In [46]:
import dlib
import dlib
import cv2
from imutils import face_utils
import argparse
from utils import *
import os
import glob
import torch

In [55]:
WALK = os.walk('/media/data1/hsm/FakeAVCeleb_PREPROCESSED/FRAMES_PNG/D/TEST/')
list_path_fake = []
for (a,b,c) in WALK:
    for _c in c:
        if '.png' in _c:
            list_path_fake.append(os.path.join(a,_c))
list_path_fake.sort()


len(list_path_fake)

16119

In [43]:
def save(path, image, jpg_quality=None, png_compression=None):
    if jpg_quality:
        cv2.imwrite(path, image, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_quality])
    elif png_compression:
        cv2.imwrite(path, image, [int(cv2.IMWRITE_PNG_COMPRESSION), png_compression])  
    else:
        cv2.imwrite(path, image)

In [40]:
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 90]
result, encimg = cv2.imencode('.jpg', img, encode_param)

In [44]:
for i in range(len(list_path_fake)):
    imgpath = list_path_fake[i]
    img = cv2.imread(imgpath)
    #display the image)
    # cv2.imshow(list_path_fake[0].replace(), img)
    # save the image in JPEG format with 85% quality
    outpath_jpeg = list_path_fake[i].replace('CLRNet','CLRNet_jpg25')
    os.makedirs(os.path.dirname(outpath_jpeg),exist_ok=True)
    save(outpath_jpeg,img,jpg_quality=25)
#     print(outpath_jpeg)

## COLLECTING THE PATHS

In [14]:
WALK = os.walk('/media/data1/mhkim/FakeAVCeleb_v1.2/MULTIMODEL_DATA/FRAMES/B_m/TRAIN/REAL/')

list_path_fake = []
for (a,b,c) in WALK:
    for _c in c:
        if '.png' in _c:
            list_path_fake.append(os.path.join(a,_c))
list_path_fake.sort()

len(list_path_fake)

61638

## Start Extract

In [15]:

def midpoint(p1, p2):
    coords = (p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2
    return [int(x) for x in coords]

def preprocessing(list_path_fake, find_main_point = False):
    dict_landmark = {}
    img_list = list_path_fake

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    for file in img_list:
        img = cv2.imread(file) #img must be cropped into face
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        if len(rects) > 0:
            for rect in rects:
                if find_main_point:
                    x = rect.left()
                    y = rect.top()
                    w = rect.right()
                    h = rect.bottom()
                    img = img[y:h, x:w]

                shape = predictor(img, rect)
                shape_np = face_utils.shape_to_np(shape).tolist()
                if find_main_point:
                    left_eye = midpoint(shape_np[36], shape_np[39])
                    right_eye = midpoint(shape_np[42], shape_np[45])
                    features = [left_eye, right_eye, shape_np[33], shape_np[48], shape_np[54]]
                    dict_landmark[file] = features
                else:
                    dict_landmark[file] = shape_np
    return dict_landmark

dict_land_ = {}
dict_land_ = preprocessing(list_path_fake)

In [10]:
len(dict_land_)

60542

In [16]:
###########temp~!!!!!!!!!!!!!!
WALK = os.walk('/media/data1/mhkim/FakeAVCeleb_v1.2/MULTIMODEL_DATA/FRAMES/B_m/TEST/REAL/')

list_path = []
for (a,b,c) in WALK:
    for _c in c:
        if '.png' in _c:
            list_path.append(os.path.join(a,_c))
list_path.sort()

print(len(list_path))

def midpoint(p1, p2):
    coords = (p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2
    return [int(x) for x in coords]

def preprocessing(list_path, find_main_point = False):
    dict_landmark = {}
    img_list = list_path

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    for file in img_list:
        img = cv2.imread(file) #img must be cropped into face
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        if len(rects) > 0:
            for rect in rects:
                if find_main_point:
                    x = rect.left()
                    y = rect.top()
                    w = rect.right()
                    h = rect.bottom()
                    img = img[y:h, x:w]

                shape = predictor(img, rect)
                shape_np = face_utils.shape_to_np(shape).tolist()
                if find_main_point:
                    left_eye = midpoint(shape_np[36], shape_np[39])
                    right_eye = midpoint(shape_np[42], shape_np[45])
                    features = [left_eye, right_eye, shape_np[33], shape_np[48], shape_np[54]]
                    dict_landmark[file] = features
                else:
                    dict_landmark[file] = shape_np
    return dict_landmark

dict_land_2 = {}
dict_land_2 = preprocessing(list_path)

8003


# SAVE DICT -> JSON

In [17]:
import json
savetitle = 'land_real_train_B'
with open(f'{savetitle}.json','w', encoding='utf-8') as make_file:
    json.dump(dict_land_,make_file,indent='\t')

In [18]:
import json
savetitle = 'land_real_test_B'
with open(f'{savetitle}.json','w', encoding='utf-8') as make_file:
    json.dump(dict_land_2,make_file,indent='\t')

### LOAD JSON (FOR CHECK THE DATA)

In [27]:
import json
with open('land_fake_train.json', 'r') as f:
    landmarks_record =  json.load(f)
len(landmarks_record)

59901

In [34]:
landmarks_record['/media/data1/mhkim/FakeAVCeleb_PREPROCESSED/FRAMES_PNG/C/TRAIN/FAKE/00001_0_1985/00018.png']

[[199, 403],
 [197, 473],
 [196, 542],
 [202, 608],
 [225, 669],
 [263, 721],
 [314, 761],
 [370, 785],
 [425, 794],
 [473, 789],
 [516, 764],
 [553, 731],
 [585, 692],
 [606, 648],
 [623, 601],
 [639, 554],
 [650, 504],
 [299, 403],
 [340, 381],
 [389, 374],
 [439, 382],
 [484, 397],
 [546, 405],
 [582, 397],
 [618, 402],
 [649, 417],
 [661, 444],
 [501, 456],
 [499, 495],
 [497, 535],
 [497, 576],
 [434, 597],
 [456, 605],
 [479, 614],
 [501, 611],
 [521, 606],
 [346, 442],
 [380, 437],
 [411, 439],
 [432, 461],
 [404, 462],
 [373, 458],
 [541, 474],
 [570, 459],
 [597, 462],
 [615, 476],
 [595, 485],
 [568, 482],
 [380, 670],
 [420, 661],
 [454, 658],
 [473, 666],
 [493, 662],
 [512, 671],
 [527, 683],
 [504, 698],
 [483, 705],
 [463, 707],
 [441, 703],
 [411, 692],
 [394, 672],
 [450, 679],
 [470, 684],
 [490, 682],
 [516, 684],
 [487, 679],
 [468, 681],
 [448, 677]]

In [108]:
temp = []
temp.append(landmarks_record[list(landmarks_record)[0]])
